# COMPLEET SDM

In [ ]:
import pandas as pd
import pyodbc
from datetime import datetime

Connectie met SDM en nieuwe SQLites:

In [ ]:
go_sales_conn = sqlite3.connect("../../../data/raw/go_sales_volledig.sqlite")
go_crm_conn = sqlite3.connect("../../../data/raw/go_crm_volledig.sqlite")
go_staff_conn = sqlite3.connect("../../../data/raw/go_staff_volledig.sqlite")


DB = {"servername": r"localhost,1433", "database": "sdm", "username": "sa", "password": "iDTyjZx7dRL4"}

sdm = pyodbc.connect(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={DB['servername']};"
    f"DATABASE={DB['database']};"
    f"UID={DB['username']};"
    f"PWD={DB['password']}"
)

DataFrames maken

In [ ]:
def create_dataframes_sql(connection, db_type):
    dictionary : dict = {}
    query : str = ""
    key : str = ""

    if (db_type == "sqlite"):
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        key = "name"
    elif (db_type == "ssms"):
        query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
        key = "TABLE_NAME"

    table_names = pd.read_sql(query, connection)

    for table in table_names[key].tolist():
        dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)

    return dictionary

go_sales_tables = create_dataframes_sql(go_sales_conn, "sqlite")
go_crm_tables = create_dataframes_sql(go_crm_conn, "sqlite")
go_staff_tables = create_dataframes_sql(go_staff_conn, "sqlite")

sdm_current = create_dataframes_sql(sdm, "ssms")

Transformen van data:

In [ ]:
go_sdm_tables = go_sales_tables | go_crm_tables | go_staff_tables # Alle drie mergen in 1 Dictionary met alle DataFrames

# DataFrames met ontbrekende/verkeerd genoemde rows updaten:
go_sdm_tables["country"]["LANGUAGE"] = go_sales_tables["country"]["LANGUAGE"]

go_sdm_tables["country"]["CURRENCY_NAME"] = go_sales_tables["country"]["CURRENCY_NAME"]

go_sdm_tables["country"] = go_sdm_tables["country"].rename(columns={'COUNTRY_EN': 'COUNTRY'})

go_sdm_tables["product_line"] = go_sales_tables["product_line"]

try:
    go_sdm_tables["retailer_headquarters"].drop('POSTAL_ZONE', axis=1, inplace=True)
except KeyError:
    print("Removal of Postal Zone in tables has already been complete.")

NameError: name 'go_sales_tables' is not defined

Mergen van Dictionary dataframes en duplicates weg halen:

In [ ]:
def merge_and_remove_duplicates(sdm_current, sdm_new):
    merged_dfs = {}
    for table in sdm_new:
        if table in sdm_current:
            combined = pd.concat([sdm_current[table], sdm_new[table]], ignore_index=True)
            new_rows_only = combined.drop_duplicates()
        else:
            new_rows_only = sdm_new[table]
        merged_dfs[table] = new_rows_only
    return merged_dfs

sdm_new = merge_and_remove_duplicates(sdm_current, go_sdm_tables)

# Dictionary in goede volgorde zetten (om inserts goed te laten werken):
dict_order = [
    'sales_territory', 
    'country', 
    'order_method', 
    'retailer_site', 
    'sales_branch', 
    'sales_staff', 
    'retailer_contact', 
    'order_header', 
    'product_line', 
    'product_type', 
    'product', 
    'order_details', 
    'return_reason', 
    'returned_item', 
    'course', 
    'satisfaction_type', 
    'satisfaction', 
    'training',
    'age_group',
    'retailer_segment',
    'retailer_headquarters',
    'retailer_type',
    'retailer'
    'sales_demographic',
    'inventory_levels',
    'forecast'
]

sdm_new = {k: sdm_new[k] for k in dict_order if k in sdm_new}

Loaden van data dat nog niet in SSMS database zit:

In [ ]:
export_cursor = sdm.cursor()

for table_name, df in sdm_new.items():
    try:
        for index, row in df.iterrows():
            columns = df.columns.tolist()

            values = []
            for col in columns:
                value = row[col]

                if pd.isna(value):
                    values.append("NULL")

                elif isinstance(value, str):
                    values.append(f"'{value.replace("'", "''")}'")

                else:
                    values.append(str(value))

            column_names = ", ".join(columns)
            value_string = ", ".join(values)
            query = f"INSERT INTO {table_name} ({column_names}) VALUES ({value_string})"

            export_cursor.execute(query)
    except pyodbc.Error as e:
        print(f"Error in table: {table_name}")
        print(f"Query: {query}")
        print(f"Error message: {e}")
        print("-" * 80)

sdm.commit()
export_cursor.close()